In [1]:
#langchain 설치 및 데이터 로드
!pip install langchain openai chromadb tiktoken

  Using cached langchain-0.1.17-py3-none-any.whl.metadata (13 kB)
  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached dataclasses_json-0.6.5-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.48-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached langsmith-0.1.52-py3-none-any.whl.metadata (13 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached fastapi-0.110.3-py3-none-any.whl.metadata (24 kB)
  Using cached uvicorn-0.29.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_api-1.24.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached opentelemetry_expor

In [1]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

In [2]:
os.environ['OPENAI_API_KEY'] = "openai api key"

In [3]:
loader = DirectoryLoader("C:/chat-gpt-prg/ch03", glob="*.txt", loader_cls=TextLoader)
documents = loader.load()
print('문서의 개수 : ', len(documents))

print('1번 문서 :', documents[1])
print('------------------------')
print('2번 문서 :', documents[38])

문서의 개수 :  57
1번 문서 : page_content="정책이름: 청년활력 프로그램 운영(청년활동지원센터 운영)\n서울시청 미래청년기획단에서 주최하는 일자리 정책은 청년활력을 촉진하기 위해 자율적인 구직활동과 사회참여, 심리/정서지원 등을 제공하여 청년안전망을 구축하는 것을 목표로 합니다. 이 정책의 지원내용은 다음과 같습니다:\n\n마음건강 지원사업: 온라인 고민상담소 'hi, there'를 운영하여 마음건강에 관련된 지원을 제공합니다.\n\n사회참여 지원사업: 청년수당 참여자를 대상으로 오리엔테이션, 자기이해 프로그램, 서울잡스 등의 지원을 통해 사회참여를 도모합니다.\n\n서울청년센터 운영 지원: 서울청년센터의 운영을 지원합니다.\n\n신사업 개발 및 기획사업: 고립청년 개발지원, 홍보 등의 신규 사업 및 기획사업을 개발합니다.\n\n이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청자격은 19세부터 39세까지의 서울 거주 청년(청년수당 참여자 포함)입니다. 다양한 연령대를 대상으로 하는 프로그램도 있으며, 학력, 전공, 취업상태, 특화분야에 대한 요건은 없습니다.\n\n신청은 https://sygc.kr/ 사이트에서 각 프로그램별로 개별적으로 접수됩니다. 신청방법, 신청절차, 심사 및 발표에 대한 세부 사항은 공고를 참조하시기 바랍니다. 제출해야 할 서류는 공고에 따라 다를 수 있습니다.\n\n이 정책은 (재)서울현대교육재단과 (사)한국디지털컨버전스협회가 운영합니다. 참고로 https://blog.naver.com/sygcyouth 사이트도 참고하실 수 있습니다." metadata={'source': 'C:\\chat-gpt-prg\\ch03\\10.txt'}
------------------------
2번 문서 : page_content='정책제목: 서울시 지역연계형 청년 창업 지원사업 넥스트 로컬\n\n서울시는 전국 협력 지자체와 함께 청년의 일자리 문제를 해결하고 지역 경제 활성화를 위해 기획된 창업 지원 프로

In [4]:
#텍스트 분할하기

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수: ', len(texts))


분할된 텍스트의 개수:  64


In [5]:
print(texts[0])

page_content='정책 이름 : 청년예술지원\n서울시청 문화예술과에서는 청년 예술인들을 위한 프로그램을 진행합니다. 이 프로그램은 활동 경력이 부족한 청년 예술인들을 대상으로 하며, 최대 1,000만원의 창작 지원금과 전문가 멘토링 및 네트워킹 프로그램을 제공합니다. 이 프로그램은 2023년 1월 1일부터 2023년 12월 31일까지 운영됩니다. 신청자는 19세에서 39세까지의 나이 조건을 충족하며, 서울에서 첫 작품 발표(오프라인)를 준비 중인 예술인(단체)이어야 합니다. 학력, 전공, 취업 상태, 특화 분야에 대한 요건은 없습니다. 신청은 온라인으로 서울문화예술지원시스템(SCAS)을 통해 진행되며, 신청 기간은 2023년 10월 4일부터 10월 28일까지입니다. 심사 및 발표는 2023년 1월 경에 이루어집니다. 프로그램에 대한 자세한 내용은 https://www.scas.kr/scas/bizinfo/detail/198에서 확인할 수 있으며, 추가 문의는 02-362-9745로 하실 수 있습니다.' metadata={'source': 'C:\\chat-gpt-prg\\ch03\\1.txt'}


In [6]:
source_lst = []
for i in range(0, len(texts)):
    source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
filtered_counts = {key:value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서 :', filtered_counts)
print('분할된 텍스트의 개수 :', len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'C:\\chat-gpt-prg\\ch03\\22.txt': 2, 'C:\\chat-gpt-prg\\ch03\\23.txt': 2, 'C:\\chat-gpt-prg\\ch03\\31.txt': 2, 'C:\\chat-gpt-prg\\ch03\\36.txt': 2, 'C:\\chat-gpt-prg\\ch03\\40.txt': 2, 'C:\\chat-gpt-prg\\ch03\\48.txt': 2, 'C:\\chat-gpt-prg\\ch03\\49.txt': 2}
분할된 텍스트의 개수 : 64


In [7]:
#ChromaDB를 이용한 검색기 사용하기

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts, embedding=embedding)

retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents('신혼부부를 위한 정책이 있어?')
print('유사도가 높은 텍스트 개수 :', len(docs))
print('--'*20)
print('유사도가 높은 텍스트 중 첫 번째 텍스트 출력 :', docs[0])
print('--'*20)

print('유사도가 높은 텍스트들의 문서 출처 :')
for doc in docs:
    print(doc.metadata['source'])

c:\Users\USER\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\USER\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


유사도가 높은 텍스트 개수 : 4
----------------------------------------
유사도가 높은 텍스트 중 첫 번째 텍스트 출력 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': 'C:\\chat-gpt-prg\\ch03\\23.txt'}
----------------------------------------
유사도가 높은 텍스트들의 문서 출처 :
C:\chat-gpt-prg\ch03\23.txt
C:\chat-gpt-prg\ch03\23.txt
C:\chat-gpt-prg\ch03\40.txt
C:\chat-gpt-prg\ch03\39.txt


In [9]:
#질문으로부터 답변 얻기
qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
    chain_type='stuff', 
    retriever=retriever,
    return_source_documents=True
)

input_text = '대출과 관련된 정책이 궁금합니다'
chatbot_response = qa_chain(input_text)
print(chatbot_response)

c:\Users\USER\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '대출과 관련된 정책이 궁금합니다', 'result': '어떤 대출 정책에 대해 궁금하신가요? 서울시에서 운영하는 학자금대출 신용회복 지원사업, 역세권청년주택 주거비지원, 그리고 신혼부부 임차보증금 지원 정책 중에서 어떤 것에 대해 더 자세히 알고 싶으신가요?', 'source_documents': [Document(page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n이 정책은 서울시 미래청년기획단이 운영하며, 자세한 내용은 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n신용회복 신청 안내: https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true\n신청사이트: https://youth.seoul.go.kr/site/main/applyLoanCredit/step1', metadata={'source': 'C:\\chat-gpt-prg\\ch03\\35.txt'}), Document(page_content='정책제목: 역세권청년주택 주거비지원\n서울시 전략주택공급과에서는 청년과 신혼부부의 주거비 부담을 경감하기 위해 임차

In [10]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response['source_documents']:
        print(source.metadata['source'])

In [11]:
input_text = '신혼부부의 신혼집 마련을 위한 정책이 있을까?'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 서울시에서는 신혼부부를 위한 정책으로 '신혼부부 임차보증금 지원' 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서 출처:
C:\chat-gpt-prg\ch03\23.txt
C:\chat-gpt-prg\ch03\23.txt
C:\chat-gpt-prg\ch03\39.txt
C:\chat-gpt-prg\ch03\29.txt


In [12]:
print(chatbot_response)

{'query': '신혼부부의 신혼집 마련을 위한 정책이 있을까?', 'result': "네, 서울시에서는 신혼부부를 위한 정책으로 '신혼부부 임차보증금 지원' 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.", 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata={'source': 'C:\\chat-gpt-